In [111]:
import torch

In [112]:
USE_CUDA=torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [113]:
from torchvision import transforms, datasets

In [114]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./data/CIFAR_10/", 
                    train = True,
                    download = True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])),
                    batch_size=64, shuffle=True
)

Files already downloaded and verified


In [117]:
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./data/CIFAR_10/", 
                    train = False,
                    download = True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])),
                    batch_size=64)

Files already downloaded and verified


# model

In [141]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [142]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        
    
    def forward(self, x):
        x = self.conv1(x)  #32x32x3
        x = F.relu(x)      #32x32x8 
        x = self.pool(x)   #16x16x8
        x = self.conv2(x)  #16x16x16
        x = F.relu(x)      #16x16x16
        x = self.pool(x)    #8x8x16
        
        x = x.view(-1, 8*8*16)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x)
        return x
        

In [143]:
import torch.nn.init as init

def wegiht_init(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.xavier_normal_(m.bias.data)

In [144]:
model = CNN().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [145]:
print(model)
print(DEVICE)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)
cuda


In [146]:
from torchsummary import summary

In [147]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
         MaxPool2d-2            [-1, 8, 16, 16]               0
            Conv2d-3           [-1, 16, 16, 16]           1,168
         MaxPool2d-4             [-1, 16, 8, 8]               0
            Linear-5                   [-1, 64]          65,600
            Linear-6                   [-1, 32]           2,080
            Linear-7                   [-1, 10]             330
Total params: 69,402
Trainable params: 69,402
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.12
Params size (MB): 0.26
Estimated Total Size (MB): 0.39
----------------------------------------------------------------


/home/opc/mlenv/lib64/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [148]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE, dtype=torch.int64)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%]\tLoss{:.6f})".format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 
                100*batch_idx/len(train_loader), loss.item()))

In [149]:
def evaluate(model, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(DEVICE), target.to(DEVICE,dtype=torch.int64)
            output = model(data)
            valid_loas = F.cross_entropy(output, target)
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    valid_loss /= len(valid_loader.dataset)
    valid_accuracy =  100 * correct / len(valid_loader.dataset)
    return valid_loss, valid_accuracy

In [151]:
'''TRAINING'''
EPOCHS=1
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    valid_loss, valid_accuracy = evaluate(model, test_loader)
    print("[EPOCH:{}],\tValidationLoss: {:.4f}, \tValidation Accuracy: {:.2f}%\n".format(
    epoch, valid_loss, valid_accuracy))

/home/opc/mlenv/lib64/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/50000 (0%]	Loss0.877830)
Train Epoch: 1 [6400/50000 (13%]	Loss0.783404)
Train Epoch: 1 [12800/50000 (26%]	Loss0.866554)
Train Epoch: 1 [19200/50000 (38%]	Loss0.663985)
Train Epoch: 1 [25600/50000 (51%]	Loss0.979047)
Train Epoch: 1 [32000/50000 (64%]	Loss1.023080)
Train Epoch: 1 [38400/50000 (77%]	Loss0.720471)
Train Epoch: 1 [44800/50000 (90%]	Loss0.752378)
[EPOCH:1],	ValidationLoss: 0.0000, 	Validation Accuracy: 65.18%



In [109]:
from torchsummary import summary

In [110]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
         MaxPool2d-2            [-1, 8, 16, 16]               0
            Conv2d-3           [-1, 16, 16, 16]           1,168
         MaxPool2d-4             [-1, 16, 8, 8]               0
            Linear-5                   [-1, 64]          65,600
            Linear-6                   [-1, 32]           2,080
            Linear-7                   [-1, 10]             330
Total params: 69,402
Trainable params: 69,402
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.12
Params size (MB): 0.26
Estimated Total Size (MB): 0.39
----------------------------------------------------------------


/home/opc/mlenv/lib64/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
